In [1]:
import pandas as pd
import folium
import numpy as np
import branca.colormap as cm
import ipywidgets as widgets
from IPython.display import display



# 데이터 로드 (엑셀 파일)
file_path = 'multi_step_forecast_results.xlsx'
sheets = pd.ExcelFile(file_path).sheet_names
data = {sheet: pd.read_excel(file_path, sheet_name=sheet) for sheet in sheets}

'''
get_average_vector(lat, lon, data): 입력받은 좌표와 데이터 좌표가 가장 가까운 노드 4개 찾고 유향 유속 벡터 계산
'''

# 주어진 위치에서 유향과 유속을 벡터로 합치는 함수
def get_average_vector(lat, lon, data):
    # 주어진 위치에서 가까운 4개 지점 찾기
    data['distance'] = np.sqrt((data['latitude'] - lat) ** 2 + (data['longitude'] - lon) ** 2)
    nearest_data = data.nsmallest(4, 'distance')
    
    # 유향과 유속의 벡터 합산
    total_dx, total_dy = 0, 0
    for _, row in nearest_data.iterrows():
        # dx, dy 계산
        dx = row['predicted_speed'] * np.cos(np.radians(row['predicted_dir']))
        dy = row['predicted_speed'] * np.sin(np.radians(row['predicted_dir']))
        total_dx += dx
        total_dy += dy
    
    # 평균 벡터 계산
    avg_dx = total_dx / len(nearest_data)
    avg_dy = total_dy / len(nearest_data)
    
    return avg_dx, avg_dy

'''
좌표 input
'''

# 위도, 경도, 속도, 방향을 입력받을 위젯
latitude_input = widgets.FloatText(description="Latitude:")
longitude_input = widgets.FloatText(description="Longitude:")


'''
근처 노드 4개의 유향 유속 벡터합 dx, dy 기반 
시간 흐름에 따른 위도 경도 차이 반영
'''

# 예상 위치 계산 함수 (dx, dy 벡터를 바탕으로)
def calculate_new_position(lat, lon, dx, dy, minutes):
    # 이동 거리 (속도 * 시간) 계산
    new_lat = lat + (dy * minutes * 20/ 111000)  # 위도는 약 111km 당 1도
    new_lon = lon + (dx * minutes * 20/ (111000 * np.cos(np.radians(lat))))  # 경도는 위도에 따라 달라짐
    return new_lat, new_lon


'''
지도 생성 
'''

# 특정 시트 데이터를 지도에 표시하는 함수
def create_map(sheet_name):
    # 시트 데이터 가져오기
    sheet_data = data[sheet_name]
    
    # 지도 초기화 (중심은 시트의 평균 위치로 설정)
    lat_center = sheet_data['latitude'].mean()
    lon_center = sheet_data['longitude'].mean()
    m = folium.Map(location=[lat_center, lon_center], zoom_start=6)
    
    # 유향과 유속을 벡터로 표시
    colormap = cm.LinearColormap(colors=['blue', 'red'], vmin=sheet_data['predicted_speed'].min(), vmax=sheet_data['predicted_speed'].max())
    colormap.caption = 'Current Speed (m/s)'

    # 유향과 유속을 벡터로 시각화
    for _, row in sheet_data.iterrows():
        start = [row['latitude'], row['longitude']]
        dx = row['predicted_speed'] * np.cos(np.radians(row['predicted_dir']))
        dy = row['predicted_speed'] * np.sin(np.radians(row['predicted_dir']))
        end = [row['latitude'] + dy * 0.01, row['longitude'] + dx * 0.01]  # 벡터 크기 조정

        # 벡터를 PolyLine으로 지도에 추가
        folium.PolyLine(locations=[start, end], color=colormap(row['predicted_speed']), weight=2, opacity=0.6).add_to(m)
        
        # 시작점을 CircleMarker로 표시
        folium.CircleMarker(location=start, radius=1, color=colormap(row['predicted_speed']), fill=True, fill_color=colormap(row['predicted_speed']), fill_opacity=0.7).add_to(m)

    # 색상 맵 추가
    m.add_child(colormap)
    
    return m

'''
10분 / 20분 / 30분 시간 지남에 따른 예측
'''

# 버튼 클릭 이벤트 함수
def show_forecast_position(b):
    lat = latitude_input.value
    lon = longitude_input.value
    
    # 유향과 유속 벡터 계산
    avg_dx_10, avg_dy_10 = get_average_vector(lat, lon, data['current'])
    avg_dx_20, avg_dy_20 = get_average_vector(lat, lon, data['10min_forecast'])
    avg_dx_30, avg_dy_30 = get_average_vector(lat, lon, data['20min_forecast'])
    
    # Folium 지도 생성
    m = create_map('10min_forecast')
    
    # 10분, 20분, 30분 뒤 예상 위치 계산
    pos_10min = calculate_new_position(lat, lon, avg_dx_10, avg_dy_10, 10)
    pos_20min = calculate_new_position(lat, lon, avg_dx_20, avg_dy_20, 20)
    pos_30min = calculate_new_position(lat, lon, avg_dx_30, avg_dy_30, 30)
    
    # 예측 위치 마커 추가
    folium.Marker(location=pos_10min, popup="10 Minutes", icon=folium.Icon(color='blue')).add_to(m)
    folium.Marker(location=pos_20min, popup="20 Minutes", icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(location=pos_30min, popup="30 Minutes", icon=folium.Icon(color='red')).add_to(m)
    
    # 유향과 유속 정보 추가 (현재 위치)
    folium.Marker(location=[lat, lon], popup="Current Position", icon=folium.Icon(color='black')).add_to(m)
    
    # 지도 출력
    display(m)

# 버튼 생성
forecast_button = widgets.Button(description="Show Forecast")

# 버튼 클릭 이벤트 연결
forecast_button.on_click(show_forecast_position)

# 위젯 표시
display(latitude_input, longitude_input, forecast_button)

FloatText(value=0.0, description='Latitude:')

FloatText(value=0.0, description='Longitude:')

Button(description='Show Forecast', style=ButtonStyle())